In [62]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn

In [86]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]

In [87]:
# Preparing the monitoring of the loop
start_time = time()
requests = 0

In [93]:
# For every year in the interval 2000-2017
for year_url in years_url:
    
    # For every page in the interval 1-4
    for page in pages:
        
        # Make a get request
        response=get('https://www.imdb.com/search/title?release_date='+year_url+'&sort=num_votes,desc&page='+page)
        
        #Pause the loop
        sleep(randint(8,15))
        
        #Monitor the requests
        requests +=1
        elapsed_time=time()-start_time
        print('Requests: {};Frequency: {} requests/s'.format(requests,requests/elapsed_time))
        clear_output(wait = True)
        
        # Throw a warning for a non-200 status code
        if response.status_code !=200:
            warn('Requests: {}; Status Code: {}',format(requests, response.status_code))
        
        #Break the loop is the number of requests are greater than expected
        if requests>62:
            warn('Number of requests are greater than expected')
            break
        
        #Parse the content of the request using BeautifulSoup
        page_html=BeautifulSoup(response.text, 'html.parser')
        
        # Select all the 50 movie containers from a single page
        mv_containers=page_html.find_all('div',class_='lister-item mode-advanced')
        
        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:
                
                #Scrape the name:
                name=container.h3.a.text
                names.append(name)
                
                #Scrape the year
                year=container.h3.find('span',class_='lister-item-year text-muted unbold').text
                years.append(year)
                
                # Scrape the IMDB rating
                rating=float (container.strong.text)
                imdb_ratings.append(rating)
                
                # Scrape the Metascore
                mscore= int (container.find('span', class_='metascore').text)
                metascores.append(mscore)
                
                # Scrape the number of votes
                vote=int (first_movie.find('span',attrs={'name':'nv'})['data-value'])
                votes.append(vote)
                
        


Requests: 66;Frequency: 0.026729324025261465 requests/s


In [104]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:72; Frequency: 0.0676720602399528 requests/s


In [105]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes})

In [106]:
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2949 entries, 0 to 2948
Data columns (total 5 columns):
imdb         2949 non-null float64
metascore    2949 non-null int64
movie        2949 non-null object
votes        2949 non-null int64
year         2949 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 115.3+ KB
None


,imdb,metascore,movie,votes,year
0,8.5,67,Gladiator,1137512,(2000)
1,8.5,80,Memento,977011,(2000)
2,8.3,55,Snatch,684879,(2000)
3,8.3,68,Requiem for a Dream,664023,(2000)
4,7.4,64,X-Men,510807,(2000)
5,7.8,73,Cast Away,448591,(2000)
6,7.6,64,American Psycho,407598,(2000)
7,7.2,62,Unbreakable,294353,(2000)
8,7.0,73,Meet the Parents,283539,(2000)
9,6.1,59,Mission: Impossible II,269700,(2000)


In [107]:
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes']]
movie_ratings.head()

,movie,year,imdb,metascore,votes
0,Gladiator,(2000),8.5,67,1137512
1,Memento,(2000),8.5,80,977011
2,Snatch,(2000),8.3,55,684879
3,Requiem for a Dream,(2000),8.3,68,664023
4,X-Men,(2000),7.4,64,510807


In [109]:
movie_ratings.loc[:,'year']=movie_ratings['year'].str[-5:-1].astype(int)

In [110]:
movie_ratings['year'].head(3)

0    2000
1    2000
2    2000
Name: year, dtype: int32

In [69]:
url="https://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1"
response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [70]:
html_soup=BeautifulSoup(response.text, 'html.parser')
html_soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>IMDb: Most Voted Titles Released 2017-01-01 to 2017-12-31 - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<lin

In [71]:
movie_containers=html_soup.find_all('div', class_='lister-item mode-advanced')
print(len(movie_containers))

50


In [72]:
first_movie=movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/?ref_=adv_li_i"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB499613450_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/?ref_=adv_li_tt">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted ">
<span class="certificate">A</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</

In [73]:
first_name=first_movie.h3.a.get_text()
first_name

'Logan'

In [80]:
first_year=first_movie.h3.find('span',class_='lister-item-year text-muted unbold').text
first_year

ValueError: invalid literal for int() with base 10: '(2017)'

In [20]:
first_rating=float(first_movie.strong.text)
first_rating

7.5

In [23]:
first_mscore=int(first_movie.find('span', class_='metascore favorable').text)
first_mscore

69

In [83]:
first_votes=int (first_movie.find('span',attrs={'name':'nv'})['data-value'])
first_votes

489295

In [32]:
first_votes=int(first_votes['data-value'])

TypeError: 'int' object is not subscriptable

In [33]:
first_votes

281194

In [34]:
# Lists to store the scraped data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Extract data from individual movie container
for container in movie_containers:

    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:

        # The name
        name = container.h3.a.text
        names.append(name)

        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)

        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))

        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))

In [36]:
names

['It',
 'Den of Thieves',
 'Isle of Dogs',
 'Jumanji: Welcome to the Jungle',
 'Book Club',
 'Beirut',
 'A Wrinkle in Time',
 'Unsane',
 'Transformers: The Last Knight',
 'Justice League',
 'Midnight Sun',
 'I Can Only Imagine',
 'The Yellow Birds',
 'Unfriended: Dark Web',
 'Sherlock Gnomes',
 'Baywatch',
 'The Mountain Between Us',
 'Spider-Man: Homecoming',
 'Life of the Party',
 'The Strangers: Prey at Night',
 'I Feel Pretty',
 'Overboard',
 'American Made',
 'Searching',
 'The Shape of Water',
 'Star Wars: Episode VIII - The Last Jedi',
 'Lady Bird',
 'Wonder Woman',
 'Zoe',
 'Dunkirk']

In [37]:


test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes})
print(test_df.info())
test_df 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
imdb         30 non-null float64
metascore    30 non-null int64
movie        30 non-null object
votes        30 non-null int64
year         30 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ KB
None


,imdb,metascore,movie,votes,year
0,7.5,69,It,281194,(I) (2017)
1,7.0,49,Den of Thieves,40163,(2018)
2,8.1,82,Isle of Dogs,41534,(2018)
3,7.0,58,Jumanji: Welcome to the Jungle,174545,(2017)
4,6.3,53,Book Club,4194,(2018)
5,6.5,69,Beirut,3776,(2018)
6,4.2,53,A Wrinkle in Time,18946,(2018)
7,6.5,63,Unsane,12325,(2018)
8,5.2,27,Transformers: The Last Knight,100492,(2017)
9,6.6,45,Justice League,259446,(2017)


['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017']

In [56]:
start_time=time()
requests=0

for _ in range (5):
    requests+=1
    sleep(randint(1,3))
    elapsed_time=time()-start_time
    print('Requests: {};Frequency: {} requests/s'.format(requests,requests/elapsed_time))
    clear_output(wait = True)

Requests: 5;Frequency: 0.49721559296391116 requests/s


In [59]:
warn("Warning Simulation")

C:\Users\sanjeev.ramasamy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Warning Simulation
  """Entry point for launching an IPython kernel.


In [61]:
#Piecing everything together
Now let's piece together everything we've done so far! In the following code cell, we start by:

Redeclaring the lists variables so they become empty again.
Preparing the monitoring of the loop.
Then, we'll:

Loop through the years_url list to vary the release_date parameter of the URL.
For each element in years_url, loop through the pages list to vary the page parameter of the URL.
Make the GET requests within the pages loop (and give the headers parameter the right value to make sure we get only English content).
Pause the loop for a time interval between 8 and 15 seconds.
Monitor each request as discussed before.
Throw a warning for non-200 status codes.
Break the loop if the number of requests is greater than expected.
Convert the response's HTML content to a BeautifulSoup object.
Extract all movie containers from this BeautifulSoup object.
Loop through all these containers.
Extract the data if a container has a Metascore.

SyntaxError: invalid syntax (<ipython-input-61-df023e7367d1>, line 2)